In [23]:
require 'sequel'

DB = Sequel.connect(adapter: 'postgres', 
                    host: 'postgres', 
                    user: 'postgres',
                    password: 'mysecretpassword',
                    database: 'adr',
                    search_path: ['mpd', 'meddra','reports'])



TB = Sequel.connect(adapter: 'postgres', 
                    host: 'postgres', 
                    user: 'postgres',
                    password: 'mysecretpassword',
                    database: 'prr',
                    search_path: ['olap_analysis', 'public']
                    )

load 'etl_prr/source_model.rb'
load 'etl_prr/extensions.rb'
load 'etl_prr/target_model.rb'


<main>:2: warning: already initialized constant DB
<main>:2: warning: previous definition of DB was here
<main>:11: warning: already initialized constant TB
<main>:11: warning: previous definition of TB was here


true

## First Schema

In [3]:
def create_dim_time_table!(db_connection,  params = {} )
  default_params = { model: nil, time_att: :vt}
  p = default_params.merge(params)
  db_connection.create_table!(:dim_time) do
    Date    :vt, primary_key: true
    String  :month
    String  :year 
  end  
  if not p[:model].nil? 
      p[:model].all.map{|item| item.public_send(p[:time_att])}.uniq.each do |vt|      
        db_connection[:dim_time].insert(
          vt: vt, 
          month: vt.to_s[0..6], 
          year: vt.to_s[0..3] 
        ) 
      end  
  end  
end 

:create_dim_time_table!

In [5]:
def create_fact_signal_table!(db_connection)
  db_connection.create_table!(:signals) do
    primary_key :id
    Date    :vt 
  end  
   
end 

def create_dim_adr_table!(db_connection)

  db_connection.create_table!(:dim_adr) do
    Integer :id, primary_key: true
    String :low_term
    String :preferred_term
    String :soc
  end  
   
end 

def create_dim_drug_table!(db_connection)

  db_connection.create_table!(:dim_drug) do
    Integer :id, primary_key: true
    String :brand_name
    String :name
    String :atc_5
    String :atc_4
    String :atc_3
    String :atc_2
    String :atc_1
  end  
   
end 

:create_dim_drug_table!

In [21]:
create_dim_time_table!(TB, model: Treport)
create_fact_signal_table!(TB)
create_dim_adr_table!(TB)
create_dim_drug_table!(TB)

In [22]:
Treport.all.each do |report|
  report.tadrs.product(report.tdrugs.select{|drug| not drug.atc.nil?}).each do |adr, drug|
    signal_id = TB[:signals].insert(vt: report.vt)
    TB[:dim_adr].insert(id: signal_id, low_term: adr.name, preferred_term: adr.preferred_name, soc: adr.soc)
    TB[:dim_drug].insert(id: signal_id, 
      brand_name: drug.brand_name,
      name: drug.name,
      atc_1: drug.atc[0],
      atc_2: drug.atc[0..2],
      atc_3: drug.atc[0..3],
      atc_4: drug.atc[0..4],
      atc_5: drug.atc[0..6]
    )  
  end  
end  

nil

## Second Schema

In [51]:
def fast_table!(db_connection, name, params = {})
  default_params = { dates: [], integers: [], strings: [] }
  p = default_params.merge(params) 
  c = 
  "db_connection.create_table!(:#{name}) do
    primary_key :id
   #{p[:dates].map{|att| " Date :#{att}"}.join("\n")}\n
   #{p[:integers].map{|att| " Integer :#{att}"}.join("\n")}\n
   #{p[:strings].map{|att| " String :#{att}"}.join("\n")}\n
   end"  
   eval c
end  

:fast_table!

In [52]:
fast_table! TB, :ext_signals, dates:[:vt],integers:[:adr_id, :drug_id]
fast_table! TB, :ext_drugs,  strings: [:brand_name,:name]+(1..5).map{|i| "atc_#{i}".to_sym }
fast_table! TB, :ext_adrs, strings: [:low_term,:preferred_term,:soc] 

In [54]:
Treport.all.each do |report|
  report.tadrs.product(report.tdrugs.select{|drug| not drug.atc.nil?}).each do |adr, drug|
    ext_adr = TB[:ext_adrs].where(low_term: adr.name).to_a
    adr_id = ext_adr.empty? ?  TB[:ext_adrs].insert(
      low_term: adr.name, 
      preferred_term: adr.preferred_name, 
      soc: adr.soc) :  ext_adr[0][:id] 
    
    ext_drug = TB[:ext_drugs].where(brand_name: drug.brand_name).to_a
    drug_id = ext_drug.empty? ? TB[:ext_drugs].insert( 
      brand_name: drug.brand_name,
      name: drug.name,
      atc_1: drug.atc[0],
      atc_2: drug.atc[0..2],
      atc_3: drug.atc[0..3],
      atc_4: drug.atc[0..4],
      atc_5: drug.atc[0..6]) : ext_drug[0][:id] 
    
    TB[:ext_signals].insert(vt: report.vt, adr_id: adr_id, drug_id: drug_id)
    
  end  
end  

[#<Treport @values={:id=>1, :source_id=>214813, :vt=>#<Date: 2013-08-21 ((2456526j,0s,0n),+0s,2299161j)>}>, #<Treport @values={:id=>71, :source_id=>399776, :vt=>#<Date: 2017-02-24 ((2457809j,0s,0n),+0s,2299161j)>}>, #<Treport @values={:id=>2, :source_id=>44434, :vt=>#<Date: 2002-06-24 ((2452450j,0s,0n),+0s,2299161j)>}>, #<Treport @values={:id=>154, :source_id=>406070, :vt=>#<Date: 2017-04-19 ((2457863j,0s,0n),+0s,2299161j)>}>, #<Treport @values={:id=>3, :source_id=>332173, :vt=>#<Date: 2015-10-21 ((2457317j,0s,0n),+0s,2299161j)>}>, #<Treport @values={:id=>72, :source_id=>251427, :vt=>#<Date: 2014-04-11 ((2456759j,0s,0n),+0s,2299161j)>}>, #<Treport @values={:id=>4, :source_id=>248516, :vt=>#<Date: 2014-01-30 ((2456688j,0s,0n),+0s,2299161j)>}>, #<Treport @values={:id=>5, :source_id=>344293, :vt=>#<Date: 2015-12-11 ((2457368j,0s,0n),+0s,2299161j)>}>, #<Treport @values={:id=>73, :source_id=>399866, :vt=>#<Date: 2017-02-24 ((2457809j,0s,0n),+0s,2299161j)>}>, #<Treport @values={:id=>6, :sour

In [46]:
[].empty?

true